In [2]:
import os
import urllib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

In [3]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC'
os.chdir(path)

csv = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\csv.csv'
f = open(csv,'w+')

txt = (r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\data_exploration\daily' + 
       r'\akr02.txt')
g = open(txt,'w+')

empty_line = ('\n')

xl_file = (path + r'\meta_files\colocation_metadata.xlsx')

In [4]:
xl = pd.ExcelFile(xl_file)

meta = xl.parse('metadata')
meta['sdate'] = meta.sdate.dt.date
meta['edate'] = meta.edate.dt.date
meta = meta.drop(meta.columns[3],axis=1)

In [5]:
n = 0
for index,rows in meta.iterrows():
    try:
        print('Pulling data from ' + str(meta.ID[n]) + '...')
        urllib.request.urlretrieve(
            'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily&sids=' +
            str(meta.ID[n]) + '&sdate=' + str(meta.sdate[n]) + '&edate=' + str(meta.edate[n]) + 
            '&elems=tave,tmax,tmin,vp,rhmax,rhmin,sr,wrun,pp,st5mx,st5mn,st15mx,' + 
            'st15mn,volts,gust,gustdir', filename=csv)
    except Exception as e:
        print('Something went wrong while pullin the data for ' + str(meta.ID[n]) + ': ' + e.args[0])
        print(empty_line)
    else:
        print('Data successfully pulled from ' + str(meta.ID[n]))
        print(empty_line)
        
    try:
        print('Loading data from ' + str(meta.ID[n]) + ' into a pandas dataframe...')
        df = pd.read_csv(csv)
        df = df.reset_index()
    except Exception as e:
        print('Something went wrong while loading data from ' + str(meta.ID[n]) + 
             'a pandas dataframe: ' + e.args[0])
        print(empty_line)
    else:
        print('Data from ' + str(meta.ID[n]) + ' succssfully loaded into a pandas dataframe')
        print(empty_line)
        
    try:
        print('Preparing the ' + str(meta.ID[n]) + ' for analysis...')
        df.columns = ['station','date','tave','tmax','tmin','vp','rhmax','rhmin','sr','wrun','pp','st5mx',
             'st5mn','st15mx','st15mn','volts','gust','gustdir']
        df['date'] = pd.to_datetime(df.date)
        df.drop(df.tail(1).index, inplace=True)
        cols = df.columns.drop(['station','date'])
        df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    except Exception as e:
        print('Something went wrong while preparing the ' + str(meta.ID[n]) + ' for analysis.')
        print(empty_line)
    else:
        print('The ' + str(meta.ID[n]) + ' is ready for analysis.')
        print(empty_line)
        
    try:
        print('Writing a statistical summary for of the raw data from' + str(meta.ID[n]) + '...')
        txt = (path + r'\data_exploration\daily\reports\nullreport_' + str(meta.ID[n]) + '.txt')
        g = open(txt,'w+') 
        title = ('Summary of data from ' + str(meta.ID[n]) + 
                 ' for the entire period of record:')
        g.write(title.center(24))
        g.write(empty_line)
        g.write(empty_line)
        g.write('Statistical Summary of Raw Data:')
        g.write(empty_line)
        g.write(empty_line)
        raw_summary = df.describe()
        g.write(str(raw_summary) + empty_line)
        g.write(empty_line)
        og_length = len(df.index)
        print('The raw dataset from ' + str(meta.ID[n]) + ' contains ' + str(og_length) + 
              ' observations.')
        print(empty_line)
    except Exception as e:
        print('Something went wrong while writing the raw summary for ' + str(meta.ID[n]) + 
             ' to a text file: ' + e.args[0])
    else:
        print('Text file for ' + str(meta.ID[n]) + ' successfully created.')
        
    try:
        print('Checking for null values reported in the average daily temperature column...')
        null_obs_string = 'Summary of null observations found in the raw dataset:'
        g.write(null_obs_string)
        g.write(empty_line)
        d = 0
        tave_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['tave'][d])):
                tave_nulls = tave_nulls + 1
            d = d + 1
        tave_string = ('There are ' + str(tave_nulls) + 
                       ' null observations of daily average temperature in the dataset.' + empty_line)
        g.write(tave_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             ' average temperature column from ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('Tave check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
              ' in the daily maximum temperature column...')
        d = 0 
        tmax_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['tmax'][d])):
                tmax_nulls = tmax_nulls + 1
            d = d + 1
        tmax_string = ('There are ' + str(tmax_nulls) + 
                       ' null observations of daily maximum temperature in the dataset.' + 
                       empty_line)
        g.write(tmax_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             'maximum temperature column from ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('Tmax check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
             ' in the daily minimum temperature column...')
        d = 0
        tmin_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['tmin'][d])):
                tmin_nulls = tmin_nulls + 1
            d = d + 1
        tmin_string = ('There are ' + str(tmin_nulls) + 
                       ' null observations of daily minimum temperature in the dataset' + 
                       empty_line)
        g.write(tmin_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' +
             'minimum temperature column from ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('Tmin check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
             ' in the daily average vapor pressure column...')
        d = 0
        vp_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['vp'][d])):
                vp_nulls = vp_nulls + 1
            d = d + 1
        vp_string = ('There are ' + str(vp_nulls) + 
                     ' null observations of daily average vapor pressure in the dataset.' + 
                     empty_line)
        g.write(vp_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             'average vapor pressure column from ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('vp check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
              ' in the daily maximum relative humidity column...')
        d = 0
        rhmax_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['rhmax'][d])):
                rhmax_nulls = rhmax_nulls + 1
            d = d + 1 
        rhmax_string = ('There are ' + str(rhmax_nulls) + 
                       ' null observations of daily maximum relative humidity in the dataset.' +
                       empty_line)
        g.write(rhmax_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             ' maximum relative humidity column from ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('rhmax check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
             ' in the daily minimum relative humidity column...')
        d = 0
        rhmin_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['rhmin'][d])):
                rhmin_nulls = rhmin_nulls + 1
            d = d + 1 
        rhmin_string = ('There are ' + str(rhmin_nulls) + 
                        ' null observations of daily minimum relative humidity in the dataset.' + 
                       empty_line)
        g.write(rhmin_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             ' minimum relative humidity column for ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('rhmin check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) +
             ' in the daily average solar radiation column...')
        d = 0
        sr_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['rhmin'][d])):
                sr_nulls = sr_nulls + 1
            d = d+1
        sr_string = ('There are ' + str(sr_nulls) + 
                    ' null observations of daily average solar radiation in the dataset.' + 
                    empty_line)
        g.write(sr_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             'average solar radiation column for ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('sr check for ' + str(meta.ID[n]) + ' was successful.')
    
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
             ' in the daily total windrun column...')
        d = 0
        wrun_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['wrun'][d])):
                wrun_nulls = wrun_nulls + 1
            d = d + 1
        wrun__string = ('There are ' + str(sr_nulls) + 
                        ' null observations of wind run reported in the dataset.' +
                       empty_line)
        g.write(wrun__string)
    except Exception as e:
        print('Something went wrong while checking for null observation in the daily ' +
             'total windrun column for ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('wrun check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) +
              'in the daily total precipitation column...')
        d = 0
        pp_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['pp'][d])):
                pp_nulls = pp_nulls + 1
            d = d + 1
        pp_string = ('There are ' + str(pp_nulls) + 
                    ' null observations of precipitation in the dataset.' + 
                    empty_line)
        g.write(pp_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             'total precipitation column for ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('precip check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
             ' in the daily maximum 5cm soil temperature column...')
        d = 0
        st5mx_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['st5mx'][d])):
                st5mx_nulls = st5mx_nulls + 1
            d = d + 1
        st5mx_string = ('There are ' + str(st5mx_nulls) +
                      ' null observations of daily maximum 5cm soil temperature in the dataset.' +
                       empty_line)
        g.write(st5mx_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             'maximum 5cm soil temperature for ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('st5 max check for ' + str(meta.ID[n]) + ' was successful.')
    
    try:
        print('Checking for null values reported in the dialy minimum 5cm soil temperature column...')
        d = 0
        st5mn_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['st5mn'][d])):
                st5mn_nulls = st5mn_nulls + 1
            d = d + 1
        st5mn_string = ('There are ' + str(st5mn_nulls) + 
                     ' null observations of daily minimum 5cm soil temperature in the dataset.' +
                       empty_line)
        g.write(st5mn_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' +
             'minimum 5cm soil temperature for ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('st5 min check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
              ' in the daily maximum 15cm soil temperature column...')
        d = 0 
        st15mx_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['st15mx'][d])):
                st15mx_nulls = st15mx_nulls + 1
            d = d + 1
        st15mx_string = ('There are ' + str(st15mx_nulls) + 
                        ' null observations of daily maximum 15cm soil temperature in the dataset.' +
                        empty_line)
        g.write(st15mx_string)
    except Exception as e:
        print('Something went wrong while checking for null observation in the daily ' + 
             'maximum 15cm soil temperature for ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('st15 max check for ' + str(meta.ID[n]) + ' was successful.')
    
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
             ' in the daily minimum 15cm soil temperature column...')
        d = 0
        st15mn_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['st15mn'][d])):
                st15mn_nulls = st15mn_nulls + 1
            d = d + 1
        st15mn_string = ('There are ' + str(st15mn_nulls) + 
                        ' null observations of daily minimum 15cm soil temperature in the dataset.' +
                        empty_line)
        g.write(st15mn_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             'minimum 15cm soil temperature for ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('st15 minimum check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
             ' in the daily average battery voltage column...')
        d = 0
        volts_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['volts'][d])):
                volts_nulls = volts_nulls + 1
            d = d + 1
        volts_string = ('There are ' + str(volts_nulls) + 
                       ' null observations of daily average battery voltage in the dataset.' +
                       empty_line)
        g.write(volts_string)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             'average battery voltage for ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('voltage check for ' + str(meta.ID[n]) + ' was successful.')
        
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
             ' in the daily maximum gust speed column...')
        d = 0
        gust_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['gust'][d])):
                gust_nulls = gust_nulls + 1
            d = d + 1
        gust_string = ('There are ' + str(gust_nulls) + 
                       ' null observations of daily maximum gust speed in the dataset.' +
                      empty_line)
        g.write(str(gust_string))
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' + 
             'average battery voltage for ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('gust check for ' + str(meta.ID[n]) + ' was successful.')
    
    try:
        print('Checking for null values reported by ' + str(meta.ID[n]) + 
             ' in the daily maximum gust direction column...')
        d = 0
        gustdir_nulls = 0
        for index,rows in df.iterrows():
            if(pd.isnull(df['gustdir'][d])):
                gust_nulls = gustdir_nulls + 1
            d = d + 1
        gustdir_string = ('There are ' + str(gustdir_nulls) + 
                          ' null observations of daily maximum gust speed direction in the dataset.' +
                         empty_line)
        g.write(gustdir_string)
        g.write(empty_line)
    except Exception as e:
        print('Something went wrong while checking for null observations in the daily ' +
             'maximum gust speed direction for' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('gust direction check for ' + str(meta.ID[n]) + ' was successful.')
        print(empty_line)
    try:
        print('Repopulating the dataset from ' + str(meta.ID[n]) + ' to find any observations '
             + ' that are entirely missing...')
        r = pd.date_range(start=str(meta.sdate[n]),end=str(meta.edate[n]),freq='D')
        missing_obs = len(r) - og_length
        missing_obs_string = ('There are ' + str(missing_obs) + 
                             ' observations entirely missing for the dataset.')
        g.write(missing_obs_string)
        g.write(empty_line)
    except Exception as e:
        print('Something went wrong while repopulating the dataset from ' + str(meta.ID[n]) + 
             ': ' + e.args[0])
    else:
        print('Dataset repopulation was successful.')
        
    try:
        print('Salvaging daily average temperature data from' + str(meta.ID[n]) 
              + ' where possible...')
        d = 0
        for index,rows in df.iterrows():
            while(pd.isnull(df['tave'][d]))and(pd.notnull(df['tmax'][d]))and(pd.notnull(df['tmin'][d])):
                df.tave = (df.tmin + df.tmax) / 2 
            d = d + 1
    except Exception as e:
        print('Someting went wrong while salvaging the daily average temperature from ' 
              + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('tave data salvage for ' + str(meta.ID[n]) + ' was successful')
        
    try:
        print('Creating a statistical summary of the repopulated dataset from ' + 
              str(meta.ID[n]) + '...')
        clean_summary = df.describe()
        g.write('Statistical summary of cleaned data:')
        g.write(empty_line)
        g.write(str(clean_summary))
        g.write(empty_line)
    except Exception as e:
        print('Something went wrong while creating a statitstical summary of the repopulated datset '
             + 'from ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('Statistical summary of repopulated dataset for ' + str(meta.ID[n]) + ' was successful.')
    try:
        print('Plotting the missing and null observations from ' + str(meta.ID[n]) + '...')
        nulls_dict = ({'tave':[tave_nulls],'tmax':[tmax_nulls],'tmin':[tmin_nulls],'vp':[vp_nulls],
              'rhmax':[rhmax_nulls],'rhmin':[rhmin_nulls],'sr':[sr_nulls],'wrun':[wrun_nulls],
              'pp':[pp_nulls],'st5mx':[st5mx_nulls],'st5mn': [st5mn_nulls],'st15mx':[st15mx_nulls],
              'st15mn':[st15mn_nulls],'volts':[volts_nulls],'gust':[gust_nulls],
              'gustdir':[gustdir_nulls]})
        
        nulls_df = pd.DataFrame(nulls_dict)
        ax = sns.catplot(data=nulls_df,kind='bar',height=11.7,aspect=2,color='b')
        ax.fig.suptitle(str(meta.ID[n]) + ' - Missing Data Plot')
        plt.ylabel('Number of null observations in the entire Dataset')
        plt.xlabel('Data types')
        plt.savefig(path + r'\data_exploration\daily\figures\nullplot_' + str(meta.ID[n]))
        plt.clf()
    except Exception as e:
        print('Something went wrong while creating a missing data plot for' + 
              str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('missing data plot for ' + str(meta.ID[n]) + ' successfully created.')
        
    try:
        ('Calculating the number of complete daily observations reported by ' + str(meta.ID[n]) +
        '...')
        df = df.dropna()
        new_length = df.index.size
        g.write('There are ' + str(new_length) + ' complete observations in the dataset.')
    except Exception as e:
        print('Something went wrong while calculating the number complete daily observations' + 
             ' reported by ' + str(meta.ID[n]) + ': ' + e.args[0])
    else:
        print('Null data reporte successful created for ' + str(meta.ID[n]))
        print(empty_line)
        print(empty_line)
    n = n + 1

        

Pulling data from AKR02...
Data successfully pulled from AKR02


Loading data from AKR02 into a pandas dataframe...
Data from AKR02 succssfully loaded into a pandas dataframe


Preparing the AKR02 for analysis...
The AKR02 is ready for analysis.


Writing a statistical summary for of the raw data fromAKR02...
The raw dataset from AKR02 contains 7450 observations.


Text file for AKR02successfully created appended.
Checking for null values reported in the average daily temperature column...
Tave check for AKR02 was successful.
Checking for null values reported by AKR02 in the daily maximum temperature column...
Tmax check for AKR02 was successful.
Checking for null values reported by AKR02 in the daily minimum temperature column...
Tmin check for AKR02 was successful.
Checking for null values reported by AKR02 in the daily average vapor pressure column...
vp check for AKR02 was successful.
Checking for null values reported by AKR02 in the daily maximum relative humidity column...
rhmax 

gust check for AVN01 was successful.
Checking for null values reported by AVN01 in the daily maximum gust direction column...
gust direction check for AVN01 was successful.


Repopulating the dataset from AVN01 to find any observations  that are entirely missing...
Dataset repopulation was successful.
Salvaging daily average temperature data fromAVN01 where possible...
tave data salvage for AVN01 was successful
Creating a statistical summary of the repopulated dataset from AVN01...
Statistical summary of repopulated dataset for AVN01 was successful.
Plotting the missing and null observations from AVN01...
missing data plot for AVN01 successfully created.
Null data reporte successful created for AVN01




Pulling data from BNV01...
Data successfully pulled from BNV01


Loading data from BNV01 into a pandas dataframe...
Data from BNV01 succssfully loaded into a pandas dataframe


Preparing the BNV01 for analysis...
The BNV01 is ready for analysis.


Writing a statistical summary for of t

precip check for BRK01 was successful.
Checking for null values reported by BRK01 in the daily maximum 5cm soil temperature column...
st5 max check for BRK01 was successful.
Checking for null values reported in the dialy minimum 5cm soil temperature column...
st5 min check for BRK01 was successful.
Checking for null values reported by BRK01 in the daily maximum 15cm soil temperature column...
st15 max check for BRK01 was successful.
Checking for null values reported by BRK01 in the daily minimum 15cm soil temperature column...
st15 minimum check for BRK01 was successful.
Checking for null values reported by BRK01 in the daily average battery voltage column...
voltage check for BRK01 was successful.
Checking for null values reported by BRK01 in the daily maximum gust speed column...
gust check for BRK01 was successful.
Checking for null values reported by BRK01 in the daily maximum gust direction column...
gust direction check for BRK01 was successful.


Repopulating the dataset from BR

vp check for CBL01 was successful.
Checking for null values reported by CBL01 in the daily maximum relative humidity column...
rhmax check for CBL01 was successful.
Checking for null values reported by CBL01 in the daily minimum relative humidity column...
rhmin check for CBL01 was successful.
Checking for null values reported by CBL01 in the daily average solar radiation column...
sr check for CBL01 was successful.
Checking for null values reported by CBL01 in the daily total windrun column...
wrun check for CBL01 was successful.
Checking for null values reported by CBL01in the daily total precipitation column...
precip check for CBL01 was successful.
Checking for null values reported by CBL01 in the daily maximum 5cm soil temperature column...
st5 max check for CBL01 was successful.
Checking for null values reported in the dialy minimum 5cm soil temperature column...
st5 min check for CBL01 was successful.
Checking for null values reported by CBL01 in the daily maximum 15cm soil temp

Tave check for COW01 was successful.
Checking for null values reported by COW01 in the daily maximum temperature column...
Tmax check for COW01 was successful.
Checking for null values reported by COW01 in the daily minimum temperature column...
Tmin check for COW01 was successful.
Checking for null values reported by COW01 in the daily average vapor pressure column...
vp check for COW01 was successful.
Checking for null values reported by COW01 in the daily maximum relative humidity column...
rhmax check for COW01 was successful.
Checking for null values reported by COW01 in the daily minimum relative humidity column...
rhmin check for COW01 was successful.
Checking for null values reported by COW01 in the daily average solar radiation column...
sr check for COW01 was successful.
Checking for null values reported by COW01 in the daily total windrun column...
wrun check for COW01 was successful.
Checking for null values reported by COW01in the daily total precipitation column...
precip

...

<Figure size 1684.8x842.4 with 0 Axes>

<Figure size 1684.8x842.4 with 0 Axes>

<Figure size 1684.8x842.4 with 0 Axes>

<Figure size 1684.8x842.4 with 0 Axes>

<Figure size 1684.8x842.4 with 0 Axes>